In [16]:
import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
import pickle

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747324939839, experiment_id='1', last_update_time=1747324939839, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747324939839, experiment_id='1', last_update_time=1747324939839, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [6]:
#client.create_experiment(name="my-cool-experiment")

In [7]:
runs = client.search_runs(
    experiment_ids=["1"],
    #filter_string="metrics.rmse < 6.35",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"Run ID: {run.info.run_id}, rmse: {run.data.metrics['rmse']}")

Run ID: ece55466f1754403a96fc52747879ecb, rmse: 6.318445793399953
Run ID: ee930eace6c54a479b0b22da22fbdb7e, rmse: 6.318445793399953
Run ID: d38295b2074849fb84828122f98f9bb3, rmse: 6.327490584667602
Run ID: e81c6903ccf848059cedbda7bd0727a7, rmse: 6.348756965511226
Run ID: b54cfc193d9c47ca8428f43a860711de, rmse: 6.357798168995213


In [9]:
run_id = "ece55466f1754403a96fc52747879ecb"
model_uri=f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1749482238903, current_stage='None', description=None, last_updated_timestamp=1749482238903, name='nyc-taxi-regressor', run_id='ece55466f1754403a96fc52747879ecb', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/ece55466f1754403a96fc52747879ecb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [10]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")


version: 5, stage: None
version: 2, stage: Staging


/tmp/ipykernel_3903/3243314508.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [37]:
model_version = 2
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_3122/2271602750.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1749138527560, current_stage='Staging', description='The NYC Taxi Predictor for Trip Duration', last_updated_timestamp=1749225162119, name='nyc-taxi-regressor', run_id='7ab4d496bf7f493180316f11467c8d2d', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/7ab4d496bf7f493180316f11467c8d2d/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost', 'stage': 'staging'}, user_id=None, version=2>

In [41]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioneed to {new_stage} on {date}."
)

<ModelVersion: aliases=[], creation_timestamp=1749138527560, current_stage='Staging', description='The model version 2 was transitioneed to Staging on 2025-06-06.', last_updated_timestamp=1749225386513, name='nyc-taxi-regressor', run_id='7ab4d496bf7f493180316f11467c8d2d', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/7ab4d496bf7f493180316f11467c8d2d/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost', 'stage': 'staging'}, user_id=None, version=2>

### 

In [11]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(url):
    df = pd.read_parquet(url)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [12]:
january_data_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet"
february_data_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"
march_data_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet"

In [13]:
df = read_dataframe(march_data_url)

In [14]:
run_id = "ece55466f1754403a96fc52747879ecb"

In [15]:
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [17]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [18]:
X_test = preprocess(df, dv)

In [20]:
target = "duration"
y_test = df[target].values

In [ ]:
#Mlflow version = 1.25.1

In [ ]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True,
)